# Zip code Scrapper

Collect Brazilian Zip codes and street names from the Internet using Selenium.

The final goal is to create a spellcheck model trained with a corpus specific to street and neighborhood names. Such spellchecker would allow us to optimize the geoprocessing workflows for the Health Department of the city of Fortaleza, Brazil. 

Here, we are going to use two complementary data sources from the internet: 1) website listing all zip codes in Brazil, and 2) the OpenStreetMap APIs. From these two sources, we are able to collect both zip codes and street names for the metropolitan region of Fortaleza, which is our region of interest. 

**Author**: Higor S. Monteiro<br>
**Date**: 16 Jun 2023<br>

In [211]:
import os
import csv
import time
import overpy
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gdf
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict

In [105]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [106]:
from sqlalchemy import create_engine
from sqlalchemy import select, insert, update, text
from sqlalchemy import Table, MetaData
from sqlalchemy import Column, DateTime, Integer, Numeric, String, Sequence, ForeignKey

In [107]:
import sys
sys.path.append(os.path.join(".."))

In [108]:
from spellgeo_cevepi.collectors.zipcode_collector import ZipCollector

## Config path

In [109]:
basepath = os.environ["HOMEPATH"]
logrpath = os.path.join(basepath, "Documents", "data", "LOGRADOUROS")

## Selenium Scraping

In [110]:
# The metropolitan region of Fortaleza (RMF) (or 'Great Fortaleza') is composed of 19 cities. 
rmf = ['fortaleza', 'trairi', 'caucaia', 'paraipaba', 'são luís do curu', 'aquiraz', 
       'cascavel', 'eusébio', 'chorozinho', 'guaiúba', 'horizonte', 'itaitinga', 'maracanaú',
       'maranguape', 'pacajus', 'pacatuba', 'pindoretama', 'são gonçalo do amarante', 'paracuru']

In [100]:
zipcol = ZipCollector()

In [101]:
zipcol.open_browser()

**Collect all city names from the state of Ceará, Brazil**

In [102]:
ce_cities = zipcol.get_cities('ce')

C:\Users\higor.monteiro\Miniconda3\envs\cevepi\lib\site-packages\bs4\element.py:1233: ResourceWarning: unclosed <socket.socket fd=2652, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('::1', 57097, 0, 0), raddr=('::1', 57094, 0, 0)>
  self._namespaces = namespaces or {}


In [105]:
print(f"No.: {len(ce_cities)}")
print(f"Examples:\n{pd.DataFrame(ce_cities, columns=['nome', 'ref']).sample(n=10)}")
ce_cities_df = pd.DataFrame(ce_cities, columns=['nome', 'ref'])
rmf_df = ce_cities_df[ce_cities_df["nome"].isin(rmf)]

No.: 802
Examples:
                        nome                                             ref
524  olho d'água do bezerril  https://listacep.com/ce/olho-dagua-do-bezerril
661        santana do acaraú       https://listacep.com/ce/santana-do-acarau
474                  mocambo                 https://listacep.com/ce/mocambo
214                 caxitoré                https://listacep.com/ce/caxitore
628                 redenção                https://listacep.com/ce/redencao
660                  santana                 https://listacep.com/ce/santana
659             santa tereza            https://listacep.com/ce/santa-tereza
70                    aurora                  https://listacep.com/ce/aurora
422              laranjeiras             https://listacep.com/ce/laranjeiras
747                 tapuiara                https://listacep.com/ce/tapuiara


**Collect all zip codes and their street names from the RMF**

In [107]:
rmf_df

,nome,ref
41,aquiraz,https://listacep.com/ce/aquiraz
205,cascavel,https://listacep.com/ce/cascavel
212,caucaia,https://listacep.com/ce/caucaia
221,chorozinho,https://listacep.com/ce/chorozinho
274,eusébio,https://listacep.com/ce/eusebio
287,fortaleza,https://listacep.com/ce/fortaleza
307,guaiúba,https://listacep.com/ce/guaiuba
319,horizonte,https://listacep.com/ce/horizonte
362,itaitinga,https://listacep.com/ce/itaitinga
443,maracanaú,https://listacep.com/ce/maracanau


In [110]:
# -- Collect ZIP from the Grande Fortaleza Region (19 cities)
rmf_info = dict()
for cur_nome, cur_ref in tqdm(rmf_df.itertuples(index=False)):
    rmf_info.update({cur_nome: []})
    
    zips = zipcol.extract_all_city(cur_ref)
    rmf_info[cur_nome] += zips

19it [05:39, 17.84s/it]


In [147]:
zipcol.close_browser()

C:\Users\higor.monteiro\Documents\projects\spellgeo-cevepi\notebooks\..\spellgeo_cevepi\collectors\zipcode_collector.py:42: ResourceWarning: unclosed <socket.socket fd=4972, family=AddressFamily.AF_INET6, type=SocketKind.SOCK_STREAM, proto=0, laddr=('::1', 64804, 0, 0), raddr=('::1', 64798, 0, 0)>
  self.driver = None


In [127]:
zips_df = [ (key, value[0], value[1], value[2]) for key in sorted(rmf_info.keys()) for value in rmf_info[key] ]
zips_df = pd.DataFrame(final_df, columns=["município", "bairro", "cep", "logradouro"])

In [57]:
zips_df.drop_duplicates(subset=["cep", "logradouro"])

,município,bairro,cep,logradouro
0,aquiraz,camará,61753975,"rua principal, s/n"
1,aquiraz,centro,61700970,avenida santos dumont 111
2,aquiraz,jacaúna,61752972,"rua principal, s/n"
3,aquiraz,justiniano serpa,61751973,"rua rufino correia assunção, s/n"
4,aquiraz,patacas,61756972,"rua josué leite de freitas, s/n"
...,...,...,...,...
22456,são luís do curu,centro,62665970,"rua antenor câmara, s/n"
22457,trairi,canaan,62692974,"rua teodorico ferreira pinto, s/n"
22458,trairi,centro,62690970,rua raimundo nonato ribeiro 257
22459,trairi,mundau,62695973,"praça da matriz, s/n"


In [241]:
zips_df.to_parquet(os.path.join(logrpath, "listacep_regiao_metropolitana_fortaleza.parquet"))
zips_df.to_csv(os.path.join(logrpath, "listacep_regiao_metropolitana_fortaleza.csv"), sep=';', encoding='latin1')

## OSM collection

Collecting street names by their Zip codes has a crucial limitation. Smaller or less developed municipalities usually have just a few zip codes even when there is a larger number of streets in the location. Therefore, it is important to complement the data by trying to extract street objects from OSM. 

In [160]:
zips_df = pd.read_parquet(os.path.join(logrpath, "listacep_regiao_metropolitana_fortaleza.parquet"))

In [31]:
def get_osm_id(place_str, name_filter):
    '''
        Get the OSM id, and the Overpass id, based on the place string parsed.
        
        Args:
        -----
            place_str:
                String. Place string to query in OSM. 
            name_filter:
                String. Name of the place to filter from the results fetched from OSM.
        Return:
        -------
            place_osmid:
                Integer.
            place_overpassid:
                Integer.
    '''
    try:
        places_df = ox.features_from_place(place_str, tags={'name': True, 'place': True, 'osmid': True}).reset_index()
    except:
        place_osmid, place_overpassid = -1, -1
        return place_osmid, place_overpassid
    places_df['name'] = places_df['name'].apply(lambda x: x.lower() if pd.notna(x) else x)
    selected_df = places_df[(places_df['element_type']=='relation') & (places_df['name']==name_filter)]
    place_osmid, place_overpassid = -1, -1
    if selected_df.shape[0]>0:
        place_osmid = selected_df['osmid'].iat[0]
        place_overpassid = place_osmid + 3600000000 # Valid only for relation type
    return place_osmid, place_overpassid

def collect_osmid(filepath, bairro_city_tupl, state_nm="ceará", country_nm="brasil"):
    '''
        ...
    
        Args:
        -----
            filepath:
                String. Filename together with the absolute path. File where results will be stored.
    '''
    place_repeated = defaultdict(lambda: False)
    # -- If did not exist before
    if not os.path.isfile(filepath):
        with open(filepath, mode="w", encoding="latin") as f:
            csv_writer = csv.writer(f, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(['place_str', 'osmid', 'overpassid', 'element_type'])
    #else:
    #    places = pd.read_csv(filepath, delimiter=";", encoding='latin', columns=['place_str'])['place_str']
    #    [ place_repeated.update({place: True}) for place in places ]
        
    osmid, overpassid = None, None
    with open(filepath, mode="a", encoding="latin") as f:
        csv_writer = csv.writer(f, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        for bairro_nm, city_nm in tqdm(bairro_city_tupl):
            current_place = f"{bairro_nm}, {city_nm}, {state_nm}, {country_nm}"
            osmid, overpassid = get_osm_id(current_place, bairro_nm)
            csv_writer.writerow([current_place, osmid, overpassid, 'relation'])
                
            # -- zoom out
            if osmid==-1 and overpassid==-1:
                current_place = f"{city_nm}, {state_nm}, {country_nm}"
                osmid, overpassid = get_osm_id(current_place, city_nm)
                csv_writer.writerow([current_place, osmid, overpassid, 'relation'])
            
        
    
def collect_streets_osm(overpassid_lst):
    '''
        ...
    '''
    results = []
    api = overpy.Overpass()
    for cur_id in tqdm(overpassid_lst):
        cur_result = api.query(f"""[timeout:900][out:json];area({cur_id});(way(area)['name']['highway'];);(._;>;);out;""")
        results += [(cur_id, r.id, r.tags['name'], r.tags) for r in cur_result.ways ]
    return results

In [32]:
final_subset = zips_df.drop_duplicates(subset=["bairro", "município"])
all_places = list(zip(final_subset['bairro'], final_subset['município']))
print(len(all_places))
all_places[:10]

354


[('camará', 'aquiraz'),
 ('centro', 'aquiraz'),
 ('jacaúna', 'aquiraz'),
 ('justiniano serpa', 'aquiraz'),
 ('patacas', 'aquiraz'),
 ('tapera', 'aquiraz'),
 ('caponga', 'cascavel'),
 ('centro', 'cascavel'),
 ('cristais', 'cascavel'),
 ('guanacés', 'cascavel')]

In [11]:
fname = os.path.join(logrpath, 'OSM_RMFortaleza_ids.csv')
#collect_osmid(fname, all_places)

100%|████████████████████████████████████████████████████████████████████████████████| 354/354 [15:58<00:00,  2.71s/it]


In [33]:
osmid_rmf = pd.read_csv(fname, delimiter=";", encoding='latin', dtype={'osmid': str, 'overpassid': str})
osmid_rmf = osmid_rmf.drop_duplicates(subset=["place_str"])

In [34]:
overpassid_lst = osmid_rmf[osmid_rmf["osmid"]!='-1']['overpassid'].tolist()

In [35]:
resul = collect_streets_osm(overpassid_lst)

100%|████████████████████████████████████████████████████████████████████████████████| 202/202 [04:53<00:00,  1.45s/it]


In [38]:
resul[1]

('3600302563',
 49013844,
 'Avenida dos Coqueiros',
 {'highway': 'residential',
  'name': 'Avenida dos Coqueiros',
  'oneway': 'no',
  'surface': 'unpaved'})

In [52]:
op_id_place = defaultdict(lambda: None, zip(osmid_rmf[osmid_rmf["osmid"]!='-1']['overpassid'], osmid_rmf[osmid_rmf["osmid"]!='-1']['place_str'])  )
op_id_osm = defaultdict(lambda: None, zip(osmid_rmf[osmid_rmf["osmid"]!='-1']['overpassid'], osmid_rmf[osmid_rmf["osmid"]!='-1']['osmid'])  )

In [161]:
# Create final database for streets in the 'Grande Fortaleza'

schema = {"origem_dado": [], "overpass_id": [], "openstreetmap_id": [], "query_osm": [], "logradouro": [], 
          "cep": [], "logradouro_overpass_id": [], "tipo_highway": [], "bairro": [], "município": [], "estado": []}

# -- OVERPASS
for tup in resul:
    overpassid, object_id, logrnm, obj = tup[0], tup[1], tup[2], tup[3]
    query_osm = op_id_place[overpassid]
    osmid = op_id_osm[overpassid]
    
    bairro_mun = [n.strip() for n in query_osm.split(",")]
    if len(bairro_mun)==4:
        schema["bairro"].append(bairro_mun[0])
        schema["município"].append(bairro_mun[1])
        schema["estado"].append('ceará')
    if len(bairro_mun)==3:
        schema["bairro"].append(np.nan)
        schema["município"].append(bairro_mun[0])
        schema["estado"].append('ceará')
    
    schema['origem_dado'].append('Overpass API')
    schema['overpass_id'].append(overpassid)
    schema["openstreetmap_id"].append(osmid)
    schema["query_osm"].append(query_osm)
    schema["logradouro"].append(logrnm.lower())
    schema["cep"].append(np.nan)
    schema["logradouro_overpass_id"].append(f"{object_id:0.0f}")
    schema["tipo_highway"].append(obj['highway'])
    
# -- LISTA CEP
for record in zips_df.to_dict(orient='records'):
    query_osm = f"{record['bairro']}, {record['município']}, ceará, brasil"
    
    schema['origem_dado'].append('https://listacep.com/')
    schema['overpass_id'].append(np.nan)
    schema["openstreetmap_id"].append(np.nan)
    schema["query_osm"].append(query_osm)
    schema["logradouro"].append(record['logradouro'].lower())
    schema["cep"].append(record['cep'])
    schema["logradouro_overpass_id"].append(np.nan)
    schema["bairro"].append(record['bairro'])
    schema["município"].append(record['município'])
    schema["estado"].append('ceará')
    schema["tipo_highway"].append(np.nan)

In [166]:
rmf_final = pd.DataFrame(schema)

In [169]:
rmf_final.to_excel(os.path.join(logrpath, "lista_logradouros_regiao_metropolitana_fortaleza.xlsx"))
rmf_final.to_parquet(os.path.join(logrpath, "lista_logradouros_regiao_metropolitana_fortaleza.parquet"))

In [153]:
#rmf_final

In [174]:
rmf_final = pd.read_parquet(os.path.join(logrpath, "lista_logradouros_regiao_metropolitana_fortaleza.parquet"))

In [176]:
rmf_final.sample(n=5)

,origem_dado,overpass_id,openstreetmap_id,query_osm,logradouro,cep,logradouro_overpass_id,tipo_highway,bairro,município,estado
22263,Overpass API,3600302588,302588,"fortaleza, ceará, brasil",rua 8,None,166672170,residential,None,fortaleza,ceará
16261,https://listacep.com/,None,None,"presidente kennedy, fortaleza, ceará, brasil",quadra h,60357270,None,None,presidente kennedy,fortaleza,ceará
20105,Overpass API,3600302588,302588,"fortaleza, ceará, brasil",rua tebas,None,158216105,residential,None,fortaleza,ceará
17590,https://listacep.com/,None,None,"siqueira, fortaleza, ceará, brasil",rua m,60732446,None,None,siqueira,fortaleza,ceará
40320,Overpass API,3605522128,5522128,"fátima, fortaleza, ceará, brasil",avenida da conquista,None,274833113,residential,fátima,fortaleza,ceará


In [252]:
rmf_unique = rmf_final.drop_duplicates(subset=["logradouro", "município", "estado"])
rmf_unique.shape

(25282, 11)

In [253]:
rmf_unique

,origem_dado,overpass_id,openstreetmap_id,query_osm,logradouro,cep,logradouro_overpass_id,tipo_highway,bairro,município,estado
0,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",rodovia santos dumont,None,23342563,motorway,None,aquiraz,ceará
1,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",avenida dos coqueiros,None,49013844,residential,None,aquiraz,ceará
2,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",rua do cangulo,None,49013845,residential,None,aquiraz,ceará
3,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",avenida aruanã,None,49013847,tertiary,None,aquiraz,ceará
5,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",rua aniquim,None,49013853,residential,None,aquiraz,ceará
...,...,...,...,...,...,...,...,...,...,...,...
22456,https://listacep.com/,None,None,"centro, são luís do curu, ceará, brasil","rua antenor câmara, s/n",62665970,None,None,centro,são luís do curu,ceará
22457,https://listacep.com/,None,None,"canaan, trairi, ceará, brasil","rua teodorico ferreira pinto, s/n",62692974,None,None,canaan,trairi,ceará
22458,https://listacep.com/,None,None,"centro, trairi, ceará, brasil",rua raimundo nonato ribeiro 257,62690970,None,None,centro,trairi,ceará
22459,https://listacep.com/,None,None,"mundau, trairi, ceará, brasil","praça da matriz, s/n",62695973,None,None,mundau,trairi,ceará


In [254]:
rmf_unique["query_geo"] = rmf_unique["logradouro"]+", "+rmf_unique["município"]+", "+"ceará, brasil"

C:\Users\higor.monteiro\AppData\Local\Temp\ipykernel_14168\598367596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rmf_unique["query_geo"] = rmf_unique["logradouro"]+", "+rmf_unique["município"]+", "+"ceará, brasil"


In [256]:
rmf_unique.merge()

,origem_dado,overpass_id,openstreetmap_id,query_osm,logradouro,cep,logradouro_overpass_id,tipo_highway,bairro,município,estado,query_geo
0,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",rodovia santos dumont,None,23342563,motorway,None,aquiraz,ceará,"rodovia santos dumont, aquiraz, ceará, brasil"
1,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",avenida dos coqueiros,None,49013844,residential,None,aquiraz,ceará,"avenida dos coqueiros, aquiraz, ceará, brasil"
2,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",rua do cangulo,None,49013845,residential,None,aquiraz,ceará,"rua do cangulo, aquiraz, ceará, brasil"
3,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",avenida aruanã,None,49013847,tertiary,None,aquiraz,ceará,"avenida aruanã, aquiraz, ceará, brasil"
5,Overpass API,3600302563,302563,"aquiraz, ceará, brasil",rua aniquim,None,49013853,residential,None,aquiraz,ceará,"rua aniquim, aquiraz, ceará, brasil"
...,...,...,...,...,...,...,...,...,...,...,...,...
22456,https://listacep.com/,None,None,"centro, são luís do curu, ceará, brasil","rua antenor câmara, s/n",62665970,None,None,centro,são luís do curu,ceará,"rua antenor câmara, s/n, são luís do curu, cea..."
22457,https://listacep.com/,None,None,"canaan, trairi, ceará, brasil","rua teodorico ferreira pinto, s/n",62692974,None,None,canaan,trairi,ceará,"rua teodorico ferreira pinto, s/n, trairi, cea..."
22458,https://listacep.com/,None,None,"centro, trairi, ceará, brasil",rua raimundo nonato ribeiro 257,62690970,None,None,centro,trairi,ceará,"rua raimundo nonato ribeiro 257, trairi, ceará..."
22459,https://listacep.com/,None,None,"mundau, trairi, ceará, brasil","praça da matriz, s/n",62695973,None,None,mundau,trairi,ceará,"praça da matriz, s/n, trairi, ceará, brasil"


In [179]:
geoquery_lst = (rmf_unique["logradouro"]+", "+rmf_unique["município"]+", "+"ceará, brasil").tolist()

In [180]:
from geopy.geocoders import Nominatim

In [181]:
geolocator = Nominatim(user_agent="teste_grande_fortaleza")

In [182]:
geoc = []
for cur_query in tqdm(geoquery_lst):
    location = geolocator.geocode(cur_query)
    if location is None:
        geoc.append((cur_query, 'none'))
    else:
        geoc.append((cur_query, location.raw, [location.latitude, location.longitude]))

#location = geolocator.geocode("175 5th Avenue NYC")

 68%|█████████████████████████████████████████████████                       | 17221/25282 [2:23:55<1:07:22,  1.99it/s]


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=vila+jos%C3%A9+de+lima%2C+fortaleza%2C+cear%C3%A1%2C+brasil&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [198]:
geoquery_lst_rest = [ geoc[n][0] for n in range(len(geoc)) if len(geoc[n])==2 ]

In [201]:
redo_geoquery_lst = geoquery_lst_rest + geoquery_lst[17221:]

In [202]:
len(redo_geoquery_lst)

9238

In [204]:
geoc_rest = []
for cur_query in tqdm(redo_geoquery_lst):
    try:
        location = geolocator.geocode(cur_query)
    except:
        location = None
    if location is None:
        geoc_rest.append((cur_query, 'none'))
    else:
        geoc_rest.append((cur_query, location.raw, [location.latitude, location.longitude]))

100%|████████████████████████████████████████████████████████████████████████████| 9238/9238 [1:19:52<00:00,  1.93it/s]


In [206]:
geo_total = geoc+geoc_rest
geoquery_coded = [ geo_total[n] for n in range(len(geo_total)) if len(geo_total[n])==3 ]

In [208]:
geoquery_coded[:5]

[('rodovia santos dumont, aquiraz, ceará, brasil',
  {'place_id': 109743305,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'osm_type': 'way',
   'osm_id': 23342563,
   'boundingbox': ['-3.9767629', '-3.9584289', '-38.515405', '-38.5132026'],
   'lat': '-3.9676087',
   'lon': '-38.5150167',
   'display_name': 'Rodovia Santos Dumont, Oiticica, Aquiraz, Região Geográfica Imediata de Fortaleza, Região Geográfica Intermediária de Fortaleza, Ceará, Região Nordeste, 61880-000, Brasil',
   'class': 'highway',
   'type': 'motorway',
   'importance': 1.0346761203690131},
  [-3.9676087, -38.5150167]),
 ('avenida dos coqueiros, aquiraz, ceará, brasil',
  {'place_id': 118771193,
   'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
   'osm_type': 'way',
   'osm_id': 49013844,
   'boundingbox': ['-3.855804', '-3.8520272', '-38.3928276', '-38.3891435'],
   'lat': '-3.8539709',
   'lon': '-38.3909867',
   'display_name': 'A

In [257]:
schema_geo = {
    "query_geo": [], 'display_name': [], 'osm_type': [], 'osm_id': [], 'lat': [], 'lon': [], 'type': []
}

for cur_geo in tqdm(geoquery_coded):
    schema_geo["query_geo"].append(cur_geo[0])
    schema_geo["display_name"].append(cur_geo[1]['display_name'])
    schema_geo["osm_type"].append(cur_geo[1]['osm_type'])
    schema_geo["osm_id"].append(f"{cur_geo[1]['osm_id']}")
    schema_geo["lat"].append(cur_geo[1]['lat'])
    schema_geo["lon"].append(cur_geo[1]['lon'])
    schema_geo["type"].append(cur_geo[1]['type'])
    
schema_geo = pd.DataFrame(schema_geo)

100%|█████████████████████████████████████████████████████████████████████████| 17885/17885 [00:00<00:00, 58445.89it/s]


In [261]:
rmf_unique1 = rmf_unique.merge(schema_geo[["query_geo", "display_name", "lat", "lon"]], on="query_geo", how="left")

In [263]:
#rmf_final.to_excel(os.path.join(logrpath, "lista_logradouros_regiao_metropolitana_fortaleza.xlsx"))
#rmf_final.to_parquet(os.path.join(logrpath, "lista_logradouros_regiao_metropolitana_fortaleza.parquet"))
rmf_unique1.to_parquet(os.path.join(logrpath, "logradouros_geo_RMFortaleza_2023.parquet"))
rmf_unique1.to_excel(os.path.join(logrpath, "logradouros_geo_RMFortaleza_2023.xlsx"))

In [242]:
schema_geo = gdf.GeoDataFrame(schema_geo, geometry=gdf.points_from_xy(schema_geo.lon, schema_geo.lat, crs="EPSG:31984"))

In [248]:
schema_geo['query_geo']

query_geo
rodovia santos dumont, aquiraz, ceará, brasil             1
rua raimundo alves, horizonte, ceará, brasil              1
rua joão gomes, horizonte, ceará, brasil                  1
rua zé janjão, horizonte, ceará, brasil                   1
rua iracilda pinheiro, horizonte, ceará, brasil           1
                                                         ..
ponte branca, fortaleza, ceará, brasil                    1
ponte azul, fortaleza, ceará, brasil                      1
rua 1125, fortaleza, ceará, brasil                        1
rua 1143, fortaleza, ceará, brasil                        1
rua raimundo nonato ribeiro 257, trairi, ceará, brasil    1
Name: count, Length: 17885, dtype: int64

In [249]:
schema_geo['osm_id'].value_counts()[:10]

osm_id
158226430    10
278054452     8
158268446     5
499578397     5
24439907      4
159340156     4
112116600     4
158216104     4
283815222     4
703432050     4
Name: count, dtype: int64

In [250]:
schema_geo[schema_geo['osm_id']=='278054452']

,query_geo,display_name,osm_type,osm_id,lat,lon,type,geometry
9576,"rua verde, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16790,"rua verde 8, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16791,"rua verde 11, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16793,"rua verde 17, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16794,"rua verde 18, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16795,"rua verde 19, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16796,"rua verde 20, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)
16803,"rua verde 38, fortaleza, ceará, brasil","Rua Verde, Vila Velha, Fortaleza, Região Geogr...",way,278054452,-3.7129169,-38.6069971,residential,POINT (-38.60700 -3.71292)


In [243]:
schema_geo.crs

<Projected CRS: EPSG:31984>
Name: SIRGAS 2000 / UTM zone 24S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 42°W and 36°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-42.0, -26.35, -36.0, 0.74)
Coordinate Operation:
- name: UTM zone 24S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [244]:
schema_geo.to_file('teste1.kml', driver='KML', index=False)

C:\Users\higor.monteiro\Miniconda3\envs\cevepi\lib\site-packages\geopandas\io\file.py:323: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(gdal_version) >= LooseVersion("3.0.0") and crs:


In [230]:
fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r',
 'KML': 'rw'}

In [225]:
import fiona
fiona.supported_drivers['KML'] = 'rw'

In [ ]:
epsg4326 wgs84

In [226]:
schema_geo.to_file('test.kml', driver='KML')

AttributeError: module 'pandas' has no attribute 'Int64Index'

In [52]:
teste = api.query(f"""[timeout:900][out:json];area({overpassid});(way(area)['name']['highway'];);(._;>;);out;""")

In [251]:
schema_geo.to_excel()

,query_geo,display_name,osm_type,osm_id,lat,lon,type,geometry
0,"rodovia santos dumont, aquiraz, ceará, brasil","Rodovia Santos Dumont, Oiticica, Aquiraz, Regi...",way,23342563,-3.9676087,-38.5150167,motorway,POINT (-38.51502 -3.96761)
1,"avenida dos coqueiros, aquiraz, ceará, brasil","Avenida dos Coqueiros, Porto das Dunas, Aquira...",way,49013844,-3.8539709,-38.3909867,residential,POINT (-38.39099 -3.85397)
2,"rua do cangulo, aquiraz, ceará, brasil","Rua do Cangulo, Vila da Prata, Conjunto Vitóri...",way,152683206,-3.9025909,-38.3757589,residential,POINT (-38.37576 -3.90259)
3,"avenida aruanã, aquiraz, ceará, brasil","Avenida Aruanã, Porto das Dunas, Aquiraz, Regi...",way,1067036304,-3.8611476,-38.3897629,primary,POINT (-38.38976 -3.86115)
4,"rua aniquim, aquiraz, ceará, brasil","Rua Aniquim, Porto das Dunas, Aquiraz, Região ...",way,49013853,-3.8526136,-38.3888197,residential,POINT (-38.38882 -3.85261)
...,...,...,...,...,...,...,...,...
17880,"avenida vale albino, s/n, pindoretama, ceará, ...","Avenida Vale Albino, Baixinha Velha, Pratiús I...",way,491206912,-4.0371744,-38.2630729,secondary,POINT (-38.26307 -4.03717)
17881,"rua francisco guilherme 41, são gonçalo do ama...","Rua Francisco Guilherme, Centro, São Gonçalo d...",way,741303895,-3.606624,-38.968819,primary,POINT (-38.96882 -3.60662)
17882,"rua marcionília sampaio 44, são gonçalo do ama...","Rua Marcionilia Sampaio, Pecém, São Gonçalo do...",way,240735476,-3.5498633,-38.8321209,tertiary,POINT (-38.83212 -3.54986)
17883,"rua teodorico ferreira pinto, s/n, trairi, cea...","Rua Teodorico Ferreira Pinto, Canaã, Trairi, R...",way,417917552,-3.2183706,-39.3637548,residential,POINT (-39.36375 -3.21837)


In [50]:


#['highway' !~ 'path']
#['highway' !~ 'motorway']
api = overpy.Overpass()
result = api.query("""
[timeout:900][out:json];
area(3600302563);
(
way(area)
['name']
['highway'];
);
(._;>;);
out;""")

reserva = """
[timeout:900][out:json];
area(3600302563);
(
way(area)
['name']
['highway']

['highway' !~ 'steps']

['highway' !~ 'motorway_link']
['highway' !~ 'raceway']
['highway' !~ 'bridleway']
['highway' !~ 'proposed']
['highway' !~ 'construction']
['highway' !~ 'elevator']
['highway' !~ 'bus_guideway']
['highway' !~ 'footway']
['highway' !~ 'cycleway']
['foot' !~ 'no']
['access' !~ 'private']
['access' !~ 'no'];
);
(._;>;);
out;"""